In [ ]:
import os
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

# 기본 설정
BASE_URL = "https://www.kw.ac.kr"
HEADERS = {"User-Agent": "Mozilla/5.0"}
SKIP_FILES = {"notice_header.png", "2025-wa-logo.webp", "commingsoon.jpg"}
SAVE_DIR = "downloaded_posters"
# 아래 문자열이 포함된 제목은 필터링
FILTER_TITLES = {"기간제 계약직원", "학사일정", "현역병", "교내 네트워크", "승강기", "직원 채용", "강사 채용", "초빙 공고", "셔틀버스", "병역", "결과", "수상", "시상", "대진", "난방", "일시 중단"}

def get_post_title(post_url):
    # 게시글 제목 추출 (<p class="title"> 내 텍스트)
    try:
        res = requests.get(post_url, headers=HEADERS, timeout=10)
        soup = BeautifulSoup(res.text, "html.parser")

        title_p = soup.select_one("p.title")
        if title_p:
            # 자식 중 strong과 span을 제외한 텍스트만 이어붙임
            title = ''.join([
                str(elem).strip() for elem in title_p.contents
                if not getattr(elem, 'name', None)
            ]).strip()
            return title
    except Exception as e:
        print(f"❌ 제목 추출 실패: {post_url}, 에러: {e}")
    return ""

def classify_keyword(title):
    # 제목 키워드 분류
    CATEGORY_KEYWORDS = [
        "대회", "모집", "행사",
        "진로", "자금", 
    ]

    CATEGORY_ALIAS = {
        "상담": "진로",
        "인턴": "진로",
        "채용": "진로",
        "자격": "진로",
        "창업": "진로",
        "아카데미": "진로",
        "캠프": "진로",
        "프로그램": "진로",
        "면접":"진로",

        "축제": "행사",
        "페스티벌": "행사",
        "포럼": "행사",
        "박람회": "행사",
        "전시회": "행사",
        "설명회": "행사",
        "세미나": "행사",
        "컨퍼런스": "행사",
        "특강": "행사",
        "강연": "행사",
        "콘서트":"행사",
        "이벤트":"행사",
        "워크숍":"행사",

        "어워드": "대회",
        "해커톤": "대회",
        "아이디어톤":"대회",
        "콘테스트": "대회",
        "오디션": "대회",
        "대전": "대회",
        "백일장": "대회",
        "콩쿠르": "대회",
        "공모": "대회",
        "챌린지": "대회",
        
        "튜터": "모집",
        "홍보대사": "모집",
        "멘토": "모집",
        "지도자": "모집",
        "서포터": "모집",
        "서포터즈": "모집",
        "크리에이터": "모집",
        "기자": "모집",
        "봉사": "모집",
        "엠버서더": "모집",
        "동아리": "모집",
        "대표단": "모집",

        "학자금":"자금",
        "대출":"자금",
        "등록금":"자금",
        "장학":"자금",
        "근로":"자금",
        "월세":"자금",
        "전세":"자금",
        "지원금":"자금",
    }

    for alias, target in CATEGORY_ALIAS.items():
        if alias in title:
            return target

    for keyword in CATEGORY_KEYWORDS:
        if keyword in title:
            return keyword

    return "기타"

def extract_images_from_post(post_url):
    # 본문 내 이미지 URL 추출
    try:
        res = requests.get(post_url, headers=HEADERS, timeout=10)
        soup = BeautifulSoup(res.text, "html.parser")
        img_tags = soup.find_all("img")
        return [urljoin(BASE_URL, img.get("src")) for img in img_tags if img.get("src")]
    except Exception as e:
        print(f"❌ 이미지 추출 실패: {post_url}, 에러: {e}")
        return []

def download_image(url, referer, duid, suffix, save_dir=SAVE_DIR):
    # 이미지 다운로드 및 저장
    os.makedirs(save_dir, exist_ok=True)
    filename = url.split("/")[-1].split("?")[0]

    if filename in SKIP_FILES:
        return

    category_dir = os.path.join(save_dir, suffix)
    os.makedirs(category_dir, exist_ok=True)

    prefixed_name = f"{duid}_{filename}"
    path = os.path.join(category_dir, prefixed_name)

    headers = {
        "User-Agent": "Mozilla/5.0",
        "Referer": referer
    }

    try:
        res = requests.get(url, headers=headers, stream=True, timeout=10)
        if res.status_code == 200:
            with open(path, "wb") as f:
                for chunk in res.iter_content(1024):
                    f.write(chunk)
            print(f"✅ 다운로드 완료: {prefixed_name}")
        else:
            print(f"❌ 다운로드 실패: {prefixed_name}, 상태 코드: {res.status_code}")
    except Exception as e:
        print(f"❌ 다운로드 에러: {prefixed_name}, 에러: {e}")

def main():
    start_duid = 50495
    end_duid = 45000

    for duid in range(start_duid, end_duid - 1, -1):
        post_url = f"{BASE_URL}/ko/life/notice.jsp?BoardMode=view&DUID={duid}"

        try:
            title = get_post_title(post_url)
            if not title:
                continue

            # 제목 필터링
            if any(skip_word in title for skip_word in FILTER_TITLES):
                continue

            print(f"📌 제목: {title}")
            suffix = classify_keyword(title)
            img_urls = extract_images_from_post(post_url)

            if not img_urls:
                print(f"⚠️ 이미지 없음: DUID {duid}")
                continue

            for img_url in img_urls:
                download_image(img_url, referer=post_url, duid=duid, suffix=suffix)

        except Exception as e:
            print(f"⚠️ 전체 처리 오류: DUID {duid}, {e}")

    print("\n🎉 전체 다운로드 완료")

if __name__ == "__main__":
    main()


📌 제목: 2025학년도 3차 다전공(복수/부/심화/연계/마이크로/학생설계융합전공) 신청 안내
📌 제목: 2025년 올해의 여성과학기술인상 후보자 추천 안내
✅ 다운로드 완료: 50489_2025_07_24_131906.png
📌 제목: 2025학년도 2학기 수강신청 공고
📌 제목: 2025 홍천 e스포츠 페스티벌
✅ 다운로드 완료: 50484_2025_07_23_114418.jpg
📌 제목: 장소사용 신청 안내(25년 8월)
📌 제목: KG-KAIROS 청년 AI 로보틱스 아카데미 6기(~8/12 마감)
✅ 다운로드 완료: 50479_2025_08_06_144529.jpg
📌 제목: [정보통신처]불법 소프트웨어, 이미지, 폰트 사용 및 배포 방지를 위한 협조 안내
📌 제목: [정보통신처]내PC지키미를 이용한 PC보안 및 알약 점검 협조 요청
📌 제목: 2025년 제19기 대한민국 바로알림단(Friends of Korea) 모집 안내
✅ 다운로드 완료: 50476_2025_07_22_130743.png
✅ 다운로드 완료: 50476_2025_07_22_130750.png
📌 제목: 19기 대한민국 바로알림단 모집
✅ 다운로드 완료: 50475_2025_07_22_113641.jpg
📌 제목: 2025 월드로보페스타
✅ 다운로드 완료: 50474_2025_07_22_113530.jpg
📌 제목: [지능형로봇혁신융합대학사업단](학점인정-추가모집) AI로봇 경진대회 참가 학생 모집합니다.
✅ 다운로드 완료: 50469_2025_07_21_115930.png
📌 제목: [대학혁신사업] 2025-1 학사경고여도 괜찮아 - 학사경고자 상담 안내 -
✅ 다운로드 완료: 50467_2025_07_21_115444.jpg
📌 제목: 글로벌 진출 지원 아카데미_일본편 참여자 모집
❌ 다운로드 실패: 50463_20250718083530_c4f705a0861325bf5bd1.jpg, 상태 코드: 403
📌 제목: e나라도움 국민소통단 모집
✅ 다운로드 완료: 50461_2025_